In [1]:
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

In [2]:
import pathlib
pathlib.Path().resolve()

WindowsPath('D:/Materi Kuliah/Semester 7/Sistem Temu Kembali Informasi/Proposal/app')

In [2]:
import pandas as pd

ms = pd.read_csv('musicFinder_2.csv')
ms.tail(5)

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
28367,zz top,old man,1970,blues,sleep away dream yesterday come care head tomo...,32,0.001880,0.139154,0.345720,0.001880,...,0.001880,0.001880,0.279757,0.615235,0.133533,0.344130,0.097279,0.342322,world/life,0.714286
28368,дэфолиант,валуев,2016,hip hop,feel wall fall feel lose feel sweetest sound f...,81,0.000810,0.239508,0.000810,0.020497,...,0.165541,0.348297,0.798549,0.787375,0.133533,0.000004,0.714551,0.939938,feelings,0.057143
28369,張露,夢斷巫山,1958,pop,remember remember thumb yeah baby time come ba...,45,0.162737,0.133445,0.315612,0.001422,...,0.001422,0.001422,0.411892,0.638259,0.909638,0.000791,0.593982,0.383364,world/life,0.885714
28370,張露,你去了那裡,1958,pop,right outside lazy summer home time soul criti...,95,0.000786,0.000786,0.113374,0.062054,...,0.215052,0.000786,0.559190,0.657180,0.937751,0.000007,0.846455,0.459443,music,0.885714
28371,彭家麗,從不喜歡孤單一個,1991,pop,lonely sidewalks silent night bring even deep ...,56,0.136184,0.001053,0.001053,0.301280,...,0.200115,0.040954,0.662082,0.698433,0.663654,0.000000,0.303380,0.503488,night/time,0.414286


In [3]:
ms = ms[ms['release_date'] >= 2000]
ms = ms[['artist_name', 'track_name','release_date', 'genre', 'lyrics', 'topic']]
ms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10412 entries, 0 to 28368
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist_name   10412 non-null  object
 1   track_name    10412 non-null  object
 2   release_date  10412 non-null  int64 
 3   genre         10412 non-null  object
 4   lyrics        10412 non-null  object
 5   topic         10412 non-null  object
dtypes: int64(1), object(5)
memory usage: 569.4+ KB


In [4]:
import re
import numpy as np

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(ms['lyrics']))
#len(norm_corpus)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(10412, 80578)

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,10402,10403,10404,10405,10406,10407,10408,10409,10410,10411
0,1.000000,0.013849,0.011936,0.029469,0.003092,0.021731,0.002323,0.012186,0.013024,0.007144,...,0.017842,0.000566,0.070585,0.004579,0.065135,0.017317,0.000000,0.000000,0.001749,0.016804
1,0.013849,1.000000,0.065282,0.030266,0.018273,0.007429,0.005785,0.027885,0.005283,0.009016,...,0.023306,0.023432,0.051465,0.018488,0.037837,0.045046,0.000000,0.008375,0.009697,0.030501
2,0.011936,0.065282,1.000000,0.028440,0.005524,0.002984,0.002807,0.020703,0.004528,0.008211,...,0.019084,0.001182,0.007234,0.010697,0.010523,0.016183,0.004019,0.013402,0.003296,0.015090
3,0.029469,0.030266,0.028440,1.000000,0.004609,0.032571,0.012148,0.017541,0.014733,0.010516,...,0.059488,0.003514,0.013628,0.023689,0.089197,0.053597,0.010653,0.000000,0.017423,0.035938
4,0.003092,0.018273,0.005524,0.004609,1.000000,0.000713,0.004166,0.018170,0.000514,0.001345,...,0.012901,0.001406,0.002143,0.001530,0.006011,0.011839,0.004459,0.000000,0.000000,0.006145


In [7]:
ms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10412 entries, 0 to 28368
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist_name   10412 non-null  object
 1   track_name    10412 non-null  object
 2   release_date  10412 non-null  int64 
 3   genre         10412 non-null  object
 4   lyrics        10412 non-null  object
 5   topic         10412 non-null  object
dtypes: int64(1), object(5)
memory usage: 569.4+ KB


In [8]:
song_list = ms['track_name'].values
song_list, song_list.shape

(array(['when your heart stops beating', 'appel local', 'leçon 3', ...,
        'cossa nostra', 'i gotsta get paid', 'валуев'], dtype=object),
 (10412,))

In [9]:
inputSong = input('Masukan lagu : ')
print("Lagu yg dicari : ", inputSong)
song_id = np.where(song_list == inputSong)[0][0]
song_id

Lagu yg dicari :  sunflower


6903

In [10]:
song_similarities = doc_sim_df.iloc[song_id].values
song_similarities

array([0.01606371, 0.03492977, 0.01759324, ..., 0.00427459, 0.01926632,
       0.02397471])

In [11]:
similar_song_id = np.argsort(-song_similarities)[1:6]
similar_song_id

array([5908, 2882, 2508, 6059, 5163], dtype=int64)

In [12]:
similar_song = song_list[similar_song_id]
similar_song

array(['wrecking ball', 'dust my broom', 'real hip hop', 'wrecking ball',
       'either way'], dtype=object)

In [13]:
def music_finder(song_name, songs = song_list, doc_sim=doc_sim_df):
    song_id = np.where(songs == song_name)[0][0]
    song_similarities = doc_sim.iloc[song_id].values
    similar_song_id = np.argsort(-song_similarities)[1:6]
    similar_song = songs[similar_song_id]
    return similar_song

In [14]:
song = input("Judul lagu: ")
result = music_finder(song_name = song, songs = song_list, doc_sim=doc_sim_df)
print("Top 5 rekomen lagu: ", result ,"\n")
detail = ms.loc[ms['track_name'].isin(result)]
sorted_detail = detail.sort_values(by='track_name')
for i in result:
    print(i)
    if i in ms['track_name'].values:
        detail = ms[ms['track_name'] == i]
        print(detail, "\n")

Top 5 rekomen lagu:  ['wrecking ball' 'dust my broom' 'real hip hop' 'wrecking ball'
 'either way'] 

wrecking ball
         artist_name     track_name  release_date  genre  \
12120      joe walsh  wrecking ball          2012  blues   
16290    miley cyrus  wrecking ball          2013    pop   
16630  mother mother  wrecking ball          2008   rock   

                                                  lyrics       topic  
12120  wind tight like hairpin trigger ready snap dri...  world/life  
16290  claw chain hearts vain jump ask kiss fell spel...     sadness  
16630  wreck hand wall fist plan reckless wreck ball ...    violence   

dust my broom
     artist_name     track_name  release_date  genre  \
8118  etta james  dust my broom          2004  blues   

                                                 lyrics     topic  
8118  dust broom inside room dust broom inside room ...  violence   

real hip hop
      artist_name    track_name  release_date    genre  \
6571    dj tomekk  re

In [15]:
song = input("Judul lagu: ")
print("Lagu yang dicari : ", song)
result = music_finder(song_name = song, songs = song_list, doc_sim=doc_sim_df)
print("Top 5 rekomen lagu: ", result ,"\n")
detail = ms.loc[ms['track_name'].isin(result)]
#sorted_detail = detail.sort_values(by='track_name')

for index, row in detail.iterrows():
    artist_name = row['artist_name']
    track_name = row['track_name']
    release_date = row['release_date']
    genre = row['genre']
    topic = row['topic']

    # Print song details
    print("{:<20} {:<20} {:<10} {:<10} {:<10}".format(artist_name, track_name, release_date, genre, topic))

Lagu yang dicari :  sunflower
Top 5 rekomen lagu:  ['wrecking ball' 'dust my broom' 'real hip hop' 'wrecking ball'
 'either way'] 

beta radio           either way           2010       pop        night/time
dj tomekk            real hip hop         2013       hip hop    violence  
etta james           dust my broom        2004       blues      violence  
joe walsh            wrecking ball        2012       blues      world/life
lee ann womack       either way           2008       country    sadness   
lil' flip            real hip hop         2018       hip hop    obscene   
miley cyrus          wrecking ball        2013       pop        sadness   
mother mother        wrecking ball        2008       rock       violence  
